In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import requests
import re
import concurrent
from time import sleep
import folium
from folium import plugins
from IPython.display import display, Math, Latex

O que este notebook ira realizar:
-------------------------------

1. Acessar o site da olx utilizando a biblioteca **BeautifulSoup**, ecnontrar a lista de preços e salvar cada item em uma lista de "Tags"
2. Encontrar os valores (municipio, estado , categoria, titulo, preco, link) de cada anuncio na lista
3. Salvar em um DataFrame do Pandas
4. Acessar cada um dos links de cada anuncio no DataFrame, no campo **href** e retira dele o CEP de cada anuncio criando um dicionario neste formato {"href":"cep"} **(Esta parte do codigo precisa ser aprimorada atualmente demora em torno de 5 à 7min para executar)**
5. Adicionar o dicionario de CEP ao dataframe de anuncios utilizando a função _.map_ do Pandas 
6. Salvar o DataFrame resultante em uma planilha Excel

In [2]:
labels = ['municipio','estado', 'categoria', 'titulo', 'preco', 'href']
valores = []
li_list = []
cep_dict = {}
#soup_list = []
url = "http://sp.olx.com.br/sao-paulo-e-regiao/zona-leste/imoveis?o="
regx = re.compile('\d{5}-\d{3}')

In [3]:
def retornaSoupOLX(url, num):
    for iter in range(num):
        r = requests.get(str(url)+str(iter))
        soup = BeautifulSoup(r.content, 'html.parser')
        for ultag in soup.find_all('ul', {'class': 'list', 'id':'main-ad-list' }):
            for litag in ultag.find_all(lambda tag: tag.name == 'li' and tag.get('class') == ['item']):
                li_list.append(litag)

In [5]:
def tiraEspecialString(string):
    return string.text.replace('\n','').replace('\t','').replace('R$ ', '').replace('.', '')

In [6]:
def tiraEspecialFloat(string):
    return string.text.replace('\n','').replace('\t','').replace('R$ ', '').replace('.', '').replace(',', '.')

In [7]:
%%time
def criaListaValores(li_list):
    for elements in li_list:
        x = BeautifulSoup(str(elements), 'html.parser')
        #encontra a parte onde estão as informaçoes no html
        info = x.find('div', {'class':'col-2'})
        #encontra a string onde estão o municipio e estado separados por virgula
        cidade_estado = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-2'}).find('p', {'class':'text detail-region'}))
        #encontra a categoria do anuncio
        categoria = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-2'}).find('p', {'class':'text detail-category'}))
        #encontra o titulo do anuncio
        titulo = tiraEspecialString(info.find('div', {'class':'OLXad-list-line-1 mb5px'}).find('h3', {'class':'OLXad-list-title mb5px'}))
        #encontra o html do preço
        preco = x.find('div', {'class':'col-3'})
        
        #trata o html do preço verificando se existe ou não
        if preco.find('p', {'class':'OLXad-list-price'}):
            preco = tiraEspecialFloat(x.find('div', {'class':'col-3'}).find('p', {'class':'OLXad-list-price'}))
        else:
            preco = 0.0
        #retorna a tag href "link do anuncio"
        href=x.find('a', {'class':'OLXad-list-link'})
        
        #retorna o conteudo do atributo href na tag href
        href = href['href'].strip(' \t\n\r')
        #separa string  onde estão o municipio
        municipio = cidade_estado.split(',',1)[1].strip(' \t\n\r')
        #separa o estado da string onde estão o municipio
        estado = cidade_estado.split(',',1)[0].strip(' \t\n\r')
        categoria = categoria.strip(' \t\n\r')
        titulo = titulo.strip(' \t\n\r')
        valores.append([municipio, estado,categoria, titulo, preco, href])

Wall time: 0 ns


In [8]:
def findCEP(url):
    r = requests.get(url)
    sleep(2)
    soup = BeautifulSoup(r.content, 'html.parser')
    cep = soup.find(lambda tag: tag.name == 'strong' and tag.get('class') == ['description'] and regx.match(tag.text.strip(' \t\n\r')) is not None)
    if cep:
        cep_dict.update({url:cep.text.strip(' \t\n\r')})
#         li_list2.append([url,cep.text.strip(' \t\n\r')])
    else:
        cep_dict.update({url:''})
#         li_list2.append('')

In [10]:
soup = retornaSoupOLX(url, 100)

Wall time: 1min 5s


In [12]:
criaListaValores(li_list)

Wall time: 14.5 s


In [13]:
df = pd.DataFrame(valores, columns=labels)

Wall time: 2.5 ms


In [14]:
list_href = list(df['href'])

Wall time: 500 µs


In [15]:
with concurrent.futures.ThreadPoolExecutor(max_workers=20) as e:
     e.map(findCEP, list_href)

Wall time: 5min 55s


In [17]:
df['cep'] = df['href'].map(cep_dict)

Wall time: 3 ms


In [18]:
df.to_excel('OLX_Scrap1.xlsx')

Wall time: 1.87 s
